# Project 3 Alternate LENET-5 CNN

Ashton Cole

AVC687

COE 379L: Software Design for Responsive Intelligent Systems

## Description

This notebook builds an alternate LENET-5 Convolutional Neural Network described in [this paper](https://arxiv.org/pdf/1807.01688.pdf).

In [1]:
import tensorflow as tf
from tensorflow.keras import optimizers, Sequential
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

2024-04-09 23:17:58.612637: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-09 23:17:58.675223: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-09 23:17:58.675279: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-09 23:17:58.678369: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-09 23:17:58.692382: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-09 23:17:58.693419: I tensorflow/core/platform/cpu_feature_guard.cc:1

## Preprocessing

In [2]:
validation_split = 0.2 # Validation split
batch_size = 32
seed = 123

image_height = 128
image_width = 128

train_dir = 'data/split/train'
test_dir = 'data/split/test'

rescale = Rescaling(scale=1.0/255)

data_train, data_validation = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    validation_split=validation_split,
    subset='both',
    seed=seed,
    image_size=(image_height, image_width),
    batch_size=batch_size)
data_test = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    seed=seed,
    image_size=(image_height, image_width),
    batch_size=batch_size)

data_train_scaled = data_train.map(lambda image, label: (rescale(image), label))
data_validation_scaled = data_validation.map(lambda image, label: (rescale(image), label))
data_test_scaled = data_test.map(lambda image, label: (rescale(image), label))

Found 17057 files belonging to 2 classes.
Using 13646 files for training.
Using 3411 files for validation.
Found 4265 files belonging to 2 classes.


## Model Construction

In [3]:
image_length = image_height * image_width * 3

model = Sequential()
# C1
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(image_height, image_width, 3)))
# S2
model.add(MaxPooling2D(pool_size=(2, 2)))
# C3
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
# S4
model.add(MaxPooling2D(pool_size=(2, 2)))
# C5
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
# S6
model.add(MaxPooling2D(pool_size=(2, 2)))
# C7
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
# S8
model.add(MaxPooling2D(pool_size=(2, 2)))
# F9
model.add(Flatten())
# F10
model.add(Dropout(0.1))
# F11
model.add(Dense(512, activation='relu'))
# Output
model.add(Dense(2, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 63, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 30, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 14, 14, 128)       0

## Model Compilation and Fitting

In [4]:
optimizer=optimizers.RMSprop(learning_rate=1e-4)
loss='sparse_categorical_crossentropy'
metrics=['accuracy']

model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics)

model.fit(
    x=data_train_scaled,
    epochs=5,
    validation_data=data_validation_scaled,
    verbose=2,
    batch_size=batch_size)

Epoch 1/5
427/427 - 91s - loss: 0.5073 - accuracy: 0.7676 - val_loss: 0.3625 - val_accuracy: 0.8634 - 91s/epoch - 213ms/step
Epoch 2/5
427/427 - 94s - loss: 0.3498 - accuracy: 0.8605 - val_loss: 0.2635 - val_accuracy: 0.8980 - 94s/epoch - 221ms/step
Epoch 3/5
427/427 - 93s - loss: 0.2712 - accuracy: 0.8926 - val_loss: 0.2231 - val_accuracy: 0.9050 - 93s/epoch - 218ms/step
Epoch 4/5
427/427 - 93s - loss: 0.2059 - accuracy: 0.9193 - val_loss: 0.2046 - val_accuracy: 0.9118 - 93s/epoch - 218ms/step
Epoch 5/5
427/427 - 93s - loss: 0.1693 - accuracy: 0.9333 - val_loss: 0.1371 - val_accuracy: 0.9405 - 93s/epoch - 218ms/step


## Model Testing

In [5]:
test_loss, test_accuracy = model.evaluate(data_test_scaled, verbose=0)
test_accuracy

0.9404454827308655

## Model Saving

In [6]:
model.save('lenet-5-modified.keras')